# Time-optimal path planning
This notebook handles standard code for the computation of time-optimal paths in windfields

In [1]:
import os
import sys
import time
import numpy as np
from math import atan, cos, sin
sys.path.extend(['/home/bastien/Documents/work/mermoz', 
                 '/home/bastien/Documents/work/mermoz/src',
                 '/home/bastien/Documents/work/mdisplay',
                 '/home/bastien/Documents/work/mdisplay/src'])

from mermoz.feedback import TargetFB
from mermoz.mdf_manager import MDFmanager
from mermoz.params_summary import ParamsSummary
from mermoz.problem import MermozProblem
from mermoz.model import ZermeloGeneralModel
from mermoz.solver import Solver
from mermoz.stoppingcond import TimedSC, DistanceSC
from mermoz.wind import DiscreteWind
from mermoz.misc import *
from mdisplay.geodata import GeoData

from solver_utils import select_wind

### Problem definition

Initial point, target, UAV airspeed

In [3]:
# Scale factor
sf = 3e6

# Initial point
x_init = sf * np.array((0., 0.))

# Target
x_target = sf * np.array((1., 0.))

# Windfield boundaries
bl = sf * np.array((-0.15, -1.15))
tr = sf * np.array((1.15, 1.15))

# UAV airspeed in m/s
v_a = 23.

# Speed factor
ssf = v_a

nx_rft = 101
ny_rft = 101
nt_rft=20

case_name = 'test1'

Set up the problem

In [4]:
# Prepare output directory
output_dir = f'/home/bastien/Documents/work/mermoz/output/example_solver_{case_name}'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Create a file manager to dump problem data
mdfm = MDFmanager()
mdfm.set_output_dir(output_dir)

# Problem coordinates
# coords = COORD_GCS
coords = COORD_CARTESIAN

# Problem wind data
from mermoz.wind import RankineVortexWind, UniformWind, DiscreteWind, RadialGaussWind
offset = sf * np.zeros(2)
omega1 = sf * (np.array([0.5, 0.8]) + offset)
omega2 = sf * (np.array([0.6, 0.2]) + offset)
omega3 = sf * (np.array([0.4, -0.5]) + offset)

vortex1 = RankineVortexWind(omega1[0], omega1[1], sf * ssf * -1., sf * 1e-1)
vortex2 = RankineVortexWind(omega2[0], omega2[1], sf * ssf * -0.8, sf * 1e-1)

const_wind = UniformWind(np.array([1., 1.]))

c1 = sf * np.array((0.5, 0.))
c2 = sf * np.array((0.5, 0.1))
c3 = sf * np.array((0.5, -0.1))

obstacle1 = RadialGaussWind(c1[0], c1[1], sf * 0.1, 1/2 * 0.2, v_a * 5.)
obstacle2 = RadialGaussWind(c2[0], c2[1], sf * 0.1, 1/2 * 0.2, v_a * 5.)
obstacle3 = RadialGaussWind(c3[0], c3[1], sf * 0.1, 1/2 * 0.2, v_a * 5.)

alty_wind = obstacle1 + obstacle2 + obstacle3 + const_wind

total_wind = alty_wind # DiscreteWind()

# total_wind.load_from_wind(alty_wind, 101, 101, bl, tr, coords='cartesian')

# Problem boundaries (default wind boundaries)
def domain(x):
    return bl[0] < x[0] < tr[0] and bl[1] < x[1] < tr[1]

# Time window upper bound
# Estimated through great circle distance + 10 percent
T = 1.1 * geodesic_distance(x_init, x_target) / v_a

# Creates the cinematic model
zermelo_model = ZermeloGeneralModel(v_a, coords=coords)
zermelo_model.update_wind(total_wind)

# Creates the navigation problem on top of the previous model
mp = MermozProblem(zermelo_model, T=T, coords=coords, autodomain=False, domain=domain)

### Solver parameters

In [5]:
u_min = DEG_TO_RAD * -45
u_max = DEG_TO_RAD * 45

nt_pmp = 1000
opti_ceil = sf * 0.01
neighb_ceil = opti_ceil/2.

solver = Solver(mp,
                x_init,
                x_target,
                T,
                u_min,
                u_max,
                output_dir,
                N_disc_init=10,
                opti_ceil=opti_ceil,
                neighb_ceil=neighb_ceil,
                n_min_opti=1,
                adaptive_int_step=False,
                N_iter=nt_pmp)

solver.log_config()
solver.setup()

Solve problem

In [6]:
t_start = time.time()
solver.solve_fancy()
t_end = time.time()
time_pmp = t_end - t_start

Shooting -45.0
Shooting -35.0
Shooting -25.0
Shooting -15.0
Shooting -5.0
Shooting 5.0
Shooting 15.0
Shooting 25.0
Shooting 35.0
Shooting 45.0
1601584.01127, 20.00
1604480.96779, 30.00
1721862.70130, -30.00
1778907.12094, -40.00
1843640.99418, 0.00
1858184.57215, -20.00
1791710.90294, 10.00
1843387.37704, -10.00
1933353.40708, 40.00
It   1 - Shoot 20.0 - Depth   1 - Crit 1643617.4, 54% - Branching LU
It   2 - Shoot 22.5 - Depth   2 - Crit 1587950.4, 52% - Branching LU
It   3 - Shoot 23.8 - Depth   3 - Crit 1551205.0, 51% - Branching LU
It   4 - Shoot 24.4 - Depth   4 - Crit 1524873.0, 50% - Branching LU
It   5 - Shoot 24.7 - Depth   5 - Crit 1505184.7, 50% - Branching LU
It   6 - Shoot 24.8 - Depth   6 -     * Pre-processing :  1.289 s
    * Processing      :  0.841 s
    * Total           :  2.130 s
No optimum found


### RFT

In [7]:
from mermoz.rft import RFT
t_start = time.time()
rft = RFT(bl, tr, T, nx_rft, ny_rft, nt_rft, mp, x_init, kernel='matlab', coords=COORD_CARTESIAN)
rft.compute()
t_end = time.time()
time_rft = t_end - t_start


tol =

   3.9000e+04

	241 steps in 6.09 seconds from 0 to 17221.9
	241 steps in 4.68 seconds from 17221.9 to 34443.7
	241 steps in 5.31 seconds from 34443.7 to 51665.6
	241 steps in 4.04 seconds from 51665.6 to 68887.5
	241 steps in 4.38 seconds from 68887.5 to 86109.4
	241 steps in 4.06 seconds from 86109.4 to 103331
	241 steps in 4.07 seconds from 103331 to 120553
	241 steps in 3.96 seconds from 120553 to 137775
	241 steps in 4.09 seconds from 137775 to 154997
	241 steps in 4.06 seconds from 154997 to 172219
	241 steps in 4.24 seconds from 172219 to 189441
	241 steps in 4.02 seconds from 189441 to 206662
	241 steps in 4 seconds from 206662 to 223884
	241 steps in 4.04 seconds from 223884 to 241106
	241 steps in 4.07 seconds from 241106 to 258328
	241 steps in 4.13 seconds from 258328 to 275550
	241 steps in 4.02 seconds from 275550 to 292772
	241 steps in 3.98 seconds from 292772 to 309994
	241 steps in 4.04 seconds from 309994 to 327216

Total execution time 81.82 seconds


### Explicit control laws

In [8]:
sc = DistanceSC(lambda x: np.linalg.norm(x - x_target), opti_ceil)
sc = TimedSC(T)
from mermoz.feedback import FixedHeadingFB, ConstantFB, TargetFB

mp.load_feedback(FixedHeadingFB(mp.model.wind, v_a, 0., mp.coords))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

mp.load_feedback(ConstantFB(0.))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

mp.load_feedback(TargetFB(mp.model.wind, v_a, x_target, COORD_CARTESIAN))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

mp.load_feedback(ConstantFB(DEG_TO_RAD * 30.))
sc = TimedSC(T/2)
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))
x_middle = np.zeros(2)
x_middle[:] = mp.trajs[-1].points[mp.trajs[-1].last_index]
mp.load_feedback(TargetFB(mp.model.wind, v_a, x_target, COORD_CARTESIAN))
mp.integrate_trajectory(x_middle, sc, int_step=T / (nt_pmp - 1))

Dump everything to output directory

In [9]:
nx = 51
ny = 101
mdfm.dump_wind(total_wind, nx=nx, ny=ny, bl=bl, tr=tr)
mdfm.dump_trajs(mp.trajs)
try:
    rft.dump_rff(output_dir)
except NameError:
    pass

ps = ParamsSummary({}, output_dir)
ps.load_from_solver(solver)
ps.add_param('bl_wind', tuple(bl))
ps.add_param('tr_wind', tuple(tr))
#ps.add_param('nx_wind', nx)
#ps.add_param('ny_wind', ny)
ps.add_param('nt_pmp', nt_pmp)
try:
    ps.add_param('pmp_time', time_pmp)
except NameError:
    pass
ps.dump()